In [1]:
%matplotlib notebook
import generator
import kd_tree
import range_tree
import test_tree_build
import test_tree_search
from structures import *
from kd_tree_visualizer import *

# Пересечение прямоугольника с множеством прямоугольников

## Постановка задачи
Пусть дано множество прямоугольников $P$, стороны которых параллельны осям координат. Нужно эффективно отвечать на запросы следующего вида: для прямоугольника $q$ (его стороны также параллельны осям координат) определить, с какими прямоугольниками из $P$ он пересекается. 

## Идея решения
Разобьём все возможные случаи пересечения $q$ с некоторым прямоугольником $p \in P$ на три (необязательно непересекающихся) множества:

* Хотя бы одна вершина $p$ лежит внутри прямоугольника запроса $q$.

<img src="images\p_in_q.png" style="width: 400px; float: midle" />

Все такие случаи можно учесть, если рассмотреть множество точек $T$, являющихся вершинами прямоугольников из $P$ и найти, какие точки из $T$ попадают в прямоугольник $q$. Таким образом, данный случай сводится к решению задачи о **нахождении всех точек, которые попадают в заданный прямоугольник**. (задача рассмотрена далее)

* Хотя бы одна вершина прямоугольника запроса $q$ лежит внутри прямоугольника $p$.

<img src="images\q_in_p.png" style="width: 400px; float: midle" />

Все такие случаи можно учесть, если для каждой вершины $q$ найти все прямоугольники, в которые она попадает. Таким образом, данный случай сводится к решению задачи о **нахождении всех прямоугольников, в которые попадает заданная точка**. (задача рассмотрена в 3-ем билете)

* Хотя бы одна сторона $p$ пересекает прямоугольник запроса $q$, но не лежит в нём концами.

<img src="images\p_intersects_q.png" style="width: 400px; float: midle" />

Все такие случаи можно учесть, если рассмотреть множество отрезков $S$, которые являются сторонами прямоугольников из $P$ и найти, какие отрезки из $S$ пересекают (но не лежат концами) прямоугольник $q$. Таким образом, данный случай сводится к решению задачи **о нахождении всех отрезков, которые пересекают заданный прямоугольник**. (задача рассмотрена в 4-ом билете)

**Ответом к задаче** будет объединение ответов к трём подзадачам.

## Нахождение множества точек, попадающих в прямоугольник запроса

Рассмотрим одномерный случай. Быстро выдавать множество точек, попадающих в отрезок, можно с помощью сбалансированного дерева поиска. Также можно с помощью отсортированного массива и бинпоиска, но в такую структуру данных, к сожалению, нельзя эффективно вставить новую точку.

Как это делать в сбалансированном дереве поиска — очевидно: идём вглубь дерева, пока не встретим узел, разделяющий концы отрезка. После этого ищем каждый конец отрезка в отдельности и добавляем к ответу все поддеревья, лежащие справа (для левого конца) и слева (для правого конца) от пути.

Такое дерево можно построить за $O(n\log n)$, она будет занимать $O(n)$ памяти, запрос будет обработан за $O(\log n + k)$, где $k$ $-$ величина ответа. Как расширить эту структуру на многомерный случай и добиться сопоставимых результатов?

## Способ 1. K-d tree

K-d дерево (short for **k-dimensional tree**) — статическая структура данных для хранения точек в k-мерном пространстве. Позволяет отвечать на запрос, какие точки лежат в данном прямоугольнике.

##### Пример:
Слева $-$ как разбивается плоскость, справа $-$ соответствующее дерево.

<img src="images\kd_tree.png" style="width: 800px; float: midle" />

### Построение
Строится это дерево следующим образом: разобьём все точки на плоскости вертикальной прямой так, чтобы слева и справа от неё было примерно поровну точек (для этого посчитаем медиану первых координат). Получим подмножества для левого и правого ребёнка. Потом построим для этих подмножеств деревья, но разбивать будем уже не вертикальной, а горизонтальной прямой (для этого посчитаем медиану вторых координат). И так далее (будем считать, что $k = 2$ (случай бОльших размерностей обрабатывается аналогично), поэтому на следующем уровне вновь будем разбивать вертикальными прямыми) пока в множестве больше одной точки.

##### Визуализатор построения:
<p>Функция *kdTreeBuildVisualize* выполняет визуализацию построения дерева для *pointsCount* точек до step шага.</p>
<p>Прямая, добавленная на текущем шаге, выделена красным.</p>

In [ ]:
pointsCount = 20
kdTreeBuildVisualize(pointsCount)

##### Реализация построения:

Пусть нам дано множество точек. Напишите функцию, которая строит 2-d tree по ним.

In [ ]:
def buildKdTree(points):
    if (len(points) == 0):
        return kd_tree.KdTree(None, 0, 0, 0, 0)

    root = buildKdNode(points, False)

    xMin = min(points, key=keyX).x
    xMax = max(points, key=keyX).x
    yMin = min(points, key=keyY).y
    yMax = max(points, key=keyY).y

    return kd_tree.KdTree(root, xMin, yMin, xMax, yMax)

def buildKdNode(points, depth): 

    # Вы получаете на вход:
    # points — список точек, соответствующий текущей ноде (объектов типа Point, имеющих поля x и y).
    # depth — флаг, показывающий, находитесь ли вы на нечетной глубине.
    #         Изначально глубина 0, так что передается False
    #         (так как нас интересует только четность глубины, удобнее передевать флаг, а не ее значение). 
    
    # Вам предлагается написать рекурсивную функцию, по входным данным строющуюю 2-d дерево
    # и возвращающую в buildKdTree указатель на корень построенного дерева.
    
    # Вершины дерева можно создавать так: kd_tree.Node().
    # Также у ноды есть поля leftChild и rightChild — указатели на левого и правого "ребенка" соответственно.
    # node.setMediana(mediana) — метод, устанавливающий значение x или y координаты (в зависимости от глубины)
    #                            разделяющей прямой, за которую отвечает эта нода (т. е медиана points).
    #                            (используется только для нод, являющихся узлами дерева)
    # node.setPoint(point) - метод, устанавливающий точку, за которую отвечает эта нода.
    #                        (используется только для нод, являющихся листами дерева)
    
    # Медиану вы можете искать с помощью функций: medianaX(points), medianaY(points),
    # возвращающих значение медианы для points по координате x и y соответственно.

    # Напишите здесь код, строящий описанное выше дерево.

    return None

test_tree_build.testBuildKdTree(buildKdTree)

##### Занимаемая память:
>*K-d дерево требует $O(n)$ памяти.*

<br>$\triangleright$<br>
<div style="padding-left:40px"> 
Высота дерева, очевидно, логарифмическая, а листьев всего $O(n)$. Поэтому будет $O(n)$ вершин, каждая занимает $O(1)$ памяти.</div>
$\triangleleft$

##### Время построения: 
>*Построение выполняется за $O(n \log n)$*.

<br>$\triangleright$<br>
<div style="padding-left:40px"> 
Время построения обозначим $T(n)$. Поиск медианы можно сделать за линейное время, поэтому легко заметить, что:

$T(n) = O(1)$ if $n = 1$.

$T(n) = O(n) + 2 \cdot T(n / 2)$, otherwise.

Решением этого рекурентного соотношения является $T(n) = O(n \log n)$.

Также стоит отметить, что можно и не искать медиану за линейное время, а просто отсортировать все точки в самом начале и дальше использовать это. В реализации попроще, асимптотика та же.</div>
$\triangleleft$

###  Запрос

Пусть нам поступил какой-то прямоугольник $R$. Нужно вернуть все точки, которые в нём лежат. Будем это делать рекурсивно, получая на вход корень дерева и сам прямоугольник $R$. Обозначим область, соответствующую вершине $v$, как $\operatorname{region(v)}$. Она будет прямоугольником, одна или более границ которого могут быть на бесконечности. $\operatorname{region(v)}$ будем считать при рекурсивном спуске. Если корень дерева является листом, то просто проверяем одну точку и при необходимости возвращаем её. Если нет, то смотрим, пересекают ли регионы детей прямоугольник $R$. Если да, то запускаемся рекурсивно от такого ребёнка. При этом, если регион полностью содержится в $R$, то можно возвращать сразу все точки из него, собрав их обходом поддерева текущей вершины. Тем самым мы, очевидно, вернём все нужные точки и только их.

##### Визуализатор выполнения запроса:

<p>Функция *kdTreeSearchVisualize* выполняет визуализацию выполнения запроса для прямоугольника *R* в дереве, построенного для *pointsCount* точек.</p>
<p>Тёмно-синяя область $-$ прямоугольник *R*.<br>
Красные области $-$ регионы, полностью входящие в *R*.<br>
Светло-синие области $-$ листья, до которых дошёл наш алгоритм.<br></p>

In [ ]:
xMin = 2
yMin = 2
xMax = 13
yMax = 14                        
pointsCount = 20
R = [xMin, yMin, xMax, yMax]   
kdTreeSearchVisualize(R, pointsCount)

##### Реализация выполнения запроса:
Пусть нам дано построенное на множестве точек 2-d tree и прямоугольник запроса. Напишите функцию, которая находит точки, попадающие в заданный прямоугольник.

In [ ]:
def pointsInRectangle(kdTree, rect):
    region = Rectangle(kdTree.xMin, kdTree.yMin, kdTree.xMax, kdTree.yMax)
    return getPoints(kdTree.root, False, region, rect)

def getPoints(node, depth, region, rect):
    
    pointsInRect = [];
    
    # Вы получаете на вход:
    # node — вершина, в которой сейчас находимся.
    #        Является объектом типа Node, поля которого описаны в задании на построение дерева.
    # depth — флаг, показывающий, находитесь ли вы на нечетной глубине.
    #         Изначально глубина 0, так что передается False.
    # region — прямогульник, за который отвечает поддерево узла node.
    # rect — прямоугольник запроса.

    # rect и region представляет из себя объекты типа Rectangle и имеет поля: xMin, yMin, xMax, yMax.
    # Эти значения являются границами прямогульника по оси x и по оси y. 
    # Создавать Rectangle можно так: Rectangle(xMin, yMin, xMax, yMax).
    # rect.include(innerRect) — метод, проверяющий, сожержит ли rect полностью innerRect.
    #                           (если содержит — возвращает True, иначе — False)
    
    # Все точки, хранящиеся в поддереве с корнем в ноде node можно получить так: kd_tree.getSubtreePoints(node).

    # Напишите здесь код рекурсивной функции, находящей точки в заданном прямоугольнике.
    
    return pointsInRect

test_tree_search.testKdTreePointsInRect(pointsInRectangle)

##### Время работы:
>*Перечисление точек в прямоугольнике выполняется за $O(\sqrt n + k)$, где $k$ — размер ответа.*

<br>$\triangleright$<br><div style="padding-left:40px"> 
Сперва заметим, что все вывод всех точек из нод дерева суммарно выполняются за $O(\text{k})$. Поэтому достаточно доказать оценку для числа рекурсивных вызовов. А рекурсивные вызовы выполняются только для тех вершин, регионы которых пересекают R, но не содержатся в нём. Такие регионы обязательно пересекают хотя бы одну сторону заданного прямоугольника. Оценим количество регионов, которые могут пересекаться произвольной вертикальной прямой. Для горизонтальной прямой это будет аналогично.

Обозначим максимально возможное количество регионов, пересекаемых какой-либо вертикальной прямой, в дереве для $n$ точек, у которого первое разбиение делается вертикальной прямой, как $Q(n)$. Рассмотрим произвольную вертикальную прямую $l$. Она будет пересекать регион корня и какого-то одного из его детей (например, левого). При этом ни один из регионов в другом (правом) поддереве пересекать она не может. Левая половина разбита ещё на 2 части горизонтальной прямой, в каждой из них примерно $n / 4$ вершин, и они хранятся в поддереве, у которого первое разбиение делается вертикальной прямой. Это даёт нам следующее соотношение:

$Q(n) = O(1)$ if $n = 1$.

$Q(n) = 2 + 2 \cdot Q(n / 4)$, otherwise.

Глубина дерева рекурсии равна: $\log_4 n = \frac{1}{2}\log_2 n$

Следовательно, $Q(n) = O(2^{\frac{1}{2}\log_2 n}) = O(\sqrt n)$ является решением. С учетом написанного выше, получаем требуемую оценку.</div>
$\triangleleft$

## Способ 2. Range tree

<img src="images\range_tree.png" style="width: 400px; float: right" />
Дадим следующее рекурсивное определение **range tree**:
* Одномерное **range tree** — обычное сбалансированное двоичное дерево поиска.
* $d$-мерное **range tree** — сбалансированное двоичное дерево поиска (по первой координате $X_1$), в каждой вершине которого дополнительно хранится $d-1$-мерное **range tree** (по остальным координатам $X_2 \times \cdots \times X_d$) для множества элементов, являющихся листами поддерева этой вершины.

*Замечание:* Используемое для range tree сбалансированное двоичное дерево поиска хранит в листах значения, для которых построено дерево, а в каждом внутреннем узле $-$ медиану значений, хранимых в листах поддерева с корнем в этом узле.


### Построение
Для начала научимся строить необходимое нам для range tree сбалансированное двоичное дерево поиска. Рекурсивно выполняем следующие действия:
* Находим медиану значений, за которые отвечает данная вершина, и сохраним ее значение в этой вершине.
* Строим аналогично левого ребенка (для точек меньших либо равных медиане) и правого (для точек больших медианы).

Продолжаем наше рекурсивное построение, пока не дойдем до вершины, которую нужно построить для одного значения. Тогда эта вершина становится листом нашего дерева и хранит то единственное значение, которое дошло до нее.

Также заметим, что если точки отсортированы, то мы сможем не тратить время на поиск медианы и построить такое сбалансированное двоичное дерево поиска за линейное время. 

Таким образом, теперь мы можем построить $d$-мерное range tree. Изначально отсортируем вершины по последней координате. Теперь построим сбалансированное двоичное дерево поиска по первой координате. Для каждого узла $v$ этого основного дерева первого уровня строим дерево второго уровня (представляющее собой $d-1$-мерное range tree для точек, являющимися листами в поддереве с корнем $v$) и сохраняем указатель на него в $v$. Это $d-1$-мерное range tree строится точно так же, как и основное дерево поиска, но уже по второй координате точек, и каждый узел в нем хранит указатель на $d-2$-мерное range tree для точек из своего поддерева, ограниченное последними $d-2$ координатами. Такое рекурсивное построение прекращается, когда останутся только последние координаты точек: они сохраняются в $1$-мерном range tree $-$ также сбалансированном двоичном дереве поиска, которое, благодаря начальной сортировке по последней координате и отсутствию вложенных деревьев, строится быстрее предыдущих.

*Замечание*: В самом глубоком дереве поиска стоит хранить также массив точек, для которых он построен, и в каждый ноде - указатель на начало блока точек из этого массива, за которые отвечает эта нода, и его длину (что возможно, так как вершины отсортированы по последней координате). Это необходимо для получения ответа на запрос в виде множества точек, при этом имея на последнем уровне вложенности дерево, занимающее линейный размер памяти.

##### Реализация построения:

Пусть нам дано множество точек. Напишите функцию, которая строит $2$-мерное range tree по ним.

In [ ]:
def buildRangeTree(points):
    points.sort(key=keyY)
    return range_tree.RangeTree(buildRangeTreeNode(points), min(points, key=keyX).x, max(points, key=keyX).x)

def buildRangeTreeNode(points):
    
    # Вы получаете на вход:
    # points — список точек, соответствующий текущей ноде (объектов типа Point).
    
    # Вам предлагается написать рекурсивную функцию, по входным данным строющуюю 2-мерное range tree
    # и возвращающую в buildRangeTree указатель на корень построенного дерева.
    
    # Вершины дерева можно создавать так: range_tree.RTNode(mediana, range_tree.buildBinarySearchTree(points)),
    # где points — список точек, рассматриваемых на момент создания ноды.
    #     mediana — медиана points по x (также является ключем текущей ноды в дереве).
    #               Находится с помощью функции medianaX(points).
    #     buildBinarySearchTree(points) — функция, строящая сбалансированное дерево поиска для points по полю y.    
    # У ноды также есть поля leftChild и rightChild — указатели на левого и правого "ребенка" соответственно.

    # Напишите здесь код, строящий описанное выше дерево.
    
    return None

test_tree_build.testBuildRangeTree(buildRangeTree)

##### Занимаемая память:
>*Range tree требует $O(n \log^{d-1} n)$ памяти.*

<br>$\triangleright$<br><div style="padding-left:40px"> 
Докажем по индукции оценку в $O(n \log^{d-1} n)$ для потребляемой памяти для структуры range tree.
* $d = 1$: в одномерном случае range tree является обычным деревом, оценка в $O(n \log^0 n) = O(n)$ верна, так как у него нет вложенных деревьев.
* $d > 1$: очевидно, что основным слагаемым в оценке потребляемой памяти будут не сами хранимые элементы, а range tree меньшей размерности, хранимые в каждой вершине, которые по индукционному предположению занимают $O(n \log^{d-2} n)$ памяти. Просуммируем размеры этих range tree (суммирование по расстоянию до вершины от корня): $\sum_{k=1}^{\log n} 2^k O(\frac{n}{2^k} \log^{d-2} \frac{n}{2^k}) = \sum_{k=1}^{\log n} O(n \log^{d-2} n) = O(n \log^{d-1} n)$. 
</div>
$\triangleleft$

##### Время построения:

>*Построение выполняется за $O(n \log^{d-1} n)$*.

<br>$\triangleright$<br>
<div style="padding-left:40px"> 

В целом доказательство этого факта аналогично доказательству для объема занимаемой памяти. Однако, стоит пояснить линейность времени построения $1$-мерного range tree. Она следует из того, что изначально мы отстортировали точки по последней координате, тем самым, позволив искать медианы в самом вложенном дереве за $O(1)$, а также из отсутствия у $1$-мерного range tree вложенных деревьев.   

</div>
$\triangleleft$

### Запрос
Запрос на выдачу точек, принадлежащих некому прямоугольнику $R$, выполняется следующим образом:<br>
Выполняем описанную в постановке задачи процедуру поиска элементов отрезка для проекции прямоугольника запроса на $X_1$.

При добавлении вершины к ответу рассмотрим текущую координату. Если она:
* последняя $-$ выдаем точки, соответствующие вершине, в качестве ответа.
* не последняя $-$ переходим к сохраненному в вершине корню range tree по следующим координатам и повторяем тот же алгоритм.

Таким образом, алгоритм сначала найдет по первой координате некоторый набор поддеревьев, потом выполнит поиск по второй координате внутри этих поддеревьев, и так далее.

##### Реализация выполнения запроса:
Пусть нам дано построенное на множестве точек $2$-мерное range tree и прямоугольник запроса. Напишите функцию, которая находит точки, попадающие в заданный прямоугольник.

In [ ]:
def pointsInRectangle(rangeTree, rect):
    result = []
    innerTrees = getTrees(rangeTree.root, rangeTree.vMin, rangeTree.vMax, rect.xMin, rect.xMax)
    for tree in innerTrees:
        result.extend(range_tree.getPoints(tree.root, tree.points, tree.vMin, tree.vMax, rect.yMin, rect.yMax))
    return result

def getTrees(node, curMin, curMax, vMin, vMax):
    
    trees = []
    
    # Вы получаете на вход:
    # node — вершина, в которой сейчас находимся. Является объектом типа RTNode, используемого в построении.
    #        Кроме указателей на детей, имеет поля: value (ключ этой ноды в дереве) и 
    #        innerTree (указатель на вложенное обычное сбалансированное дерево поиска по y).
    # curMin, curMax — левая и правая границы диапозона значений x, за который отвечает текущая нода, соответственно. 
    # vMin, vMax — левая и правая границы отрезка запроса по x на данной ноде, соответственно.
    
    # Напишите рекурсивную функцию, которая в итоге вернет список тех вложенных деревьев, в которых хранятся точки,
    # имеющие координату x, принадлежащую (нестрого) [rect.xMin, rect.xMax]. Количество возвращенных деревьев должно
    # быть минимальное возможное. 
    
    return trees

test_tree_search.testRangeTreePointsInRect(pointsInRectangle)

##### Время работы:
>*Перечисление точек в прямоугольнике выполняется за $O(\log^d n + k)$, где k $-$ количество точек ответа.*

<br>$\triangleright$<br><div style="padding-left:40px"> 
Фаза алгоритма, обрабатывающая одну координату, может выдать $O(\log n)$ поддеревьев высотой $O(\log n)$, каждое из которых будет обработано фазой по следующей координате, и т.д. Таким образом, время запроса $-$ $O(\log^d n + k)$
</div>
$\triangleleft$

### Cascading

**Cascading** $-$ оптимизация, позволяющая снизить асимптотику запроса в range tree до $O(\log^{d - 1} n + k)$.

##### Описание:

Рассмотрим две упорядоченных последовательности $S_1$ и $S_2$, причем $S_2 \subset S_1$. Пусть стоит задача выдать элементы некоторого отрезка, принадлежащие этим последовательностям. Очевидно, это можно сделать бинпоиском для каждой последовательности отдельно за $O(\log n + k)$, где $k$ $-$ размер ответа; тем не менее, это решение никак не использует тот факт, что $S_2 \subset S_1$.
Теперь, зная, что $S_2 \subset S_1$, для каждого элемента $a_1 \in S_1$ будем хранить ссылку на минимальный элемент $a_2 \in S_2$, такой, что $a_2 \geq a_1$. В таком случае мы можем избавиться от двоичного поиске по $S_2$: после поиска по $S_1$, мы можем перейти по ссылке, запомненной для первого элемента ответа из $S_1$, и выдавать элементы из $S_2$, пока они лежат в отрезке запроса. Таким образом, запрос для второй последовательности может быть осуществлён за $O(1 + k)$.
<img src="images\cascading.png" style="width: 400px; float: middle" />

##### Layered range tree:

<p>Рассмотрим $d$-мерное range tree и запрос, выполненный до $(d-1)\text{-ой}$ координаты. На последнем шаге мы выбрали набор вершин, поддеревья которых будут обработаны следующим шагом алгоритма (уже по последней координате). Всегда существует поддерево, содержащее в себе все эти вершины (корнем этого поддерева будет последняя общая вершина в путях от корня дерева к левому и правому концам отрезка запроса). Корень этого поддерева назовём $v_{split}$.
Вместо хранения деревьев, мы будем хранить массивы вершин, отсортированные по последней координате. Заметим, что множества вершин, соответствующие правому и левому детям некоторой вершины, являются подмножествами множества вершин, соответствующему их родителю.</p>
<p>Используем описанный факт, чтобы применить cascading. В массивах вместе с вершинами будем хранить ссылку на минимальный элемент массива в левом ребёнке, больший или равный текущему элементу массива; аналогично будем хранить ссылки на элементы массива в правом ребёнке текущей вершины. Range tree с такой оптимизацией называют **layered range tree**.</p>
<p>Для осуществления запроса найдём двоичным поиском в массиве, хранящемся в $v_{split}$, левую границу множества элементов, лежащих в отрезке запроса. Далее, при спуске по дереву (при поиске по предпоследней координате), мы можем поддерживать левую границу текущего поддерева (границу по последней координате), используя сохранённые ссылки. При обработке поддерева по предпоследней координате, используя сохранённый в корне поддерева массив вершин и ссылку на левую границу ответа, мы можем выдать ответ за $O(1 + k_v)$, где $k_v$ $-$ размер ответа в поддереве вершины v.
Таким образом, мы избавились от поиска по дереву на последнем уровне range tree, сократив время работы до $O(\log^{d-1} n + k)$.</p>

##### Пример:

Точки, для которых построено дерево:<br>
$(2, 19)$, $(7, 10)$, $(12, 3)$, $(17, 62)$, $(21, 49)$, $(41, 95)$, $(58, 59)$, $(93, 70)$, $(5, 80)$, $(8, 37)$, $(15, 99)$, $(33, 30)$, $(52, 23)$, $(67, 89)$

<img src="images\layered_range_tree.png" style="width: 1000px; float: midle" />